# Import required module

In [11]:
import os
import json

# Declare variables

In [12]:
CURRENT_PATH = os.getcwd()
path: str = CURRENT_PATH
source_name: str = "source.json"
target_name: str = "target.json"

# Declare functions


In [13]:
def extract_json_data(path: str, file_name: str):
    if path is None or path == "":
        raise ValueError("path variable cannot be null or empty")
    elif file_name is None or file_name == "":
        raise ValueError("file_name variable cannot be null or empty")
    else:
        with open(path+"/"+file_name, mode="r") as file:
            data = json.load(file)
            return data
    

In [14]:
def log(operation, value, parent_key, after_key):
    return {
        "operation": operation,
        "value": value,
        "parent_key": parent_key,
        "after_key": after_key
    }

In [15]:
def get_key_list(data, parent=[]):
    key_list = []
    for key, value in data.items():
        key_list.append(".".join(parent)+f".{key}" if len(parent) > 0 else key)
        if type(value) is dict:
            add_parent = parent+[key]
            key_list.extend(get_key_list(value, add_parent))
    return key_list

In [16]:
def get_values(data, target_key, exclude_dict_value=False):
    separated_scheme = target_key.split(".")
    for key in separated_scheme:
        data = data[key]
        
    if exclude_dict_value:
        if type(data) is dict:
            return None
        return {
        separated_scheme[-1]: data
    }
    else:
        return {
        separated_scheme[-1]: data
    }

In [17]:
def get_after_key(data, target_key):
    parent_key = target_key.rsplit(".", 1)[0]
    relevant_key = [key for key in data if parent_key in key and key.count(".") == target_key.count(".")]
    after_key = relevant_key[relevant_key.index(target_key)-1].split(".")[-1] if relevant_key.index(target_key) > 0 else None
    return after_key


In [18]:
def get_different_value(source_value, target_value):
    if source_value != target_value:
        return target_value
    else:
        return None


In [19]:
def get_operation(source_data, target_data):
    # Define output list
    output = []

    # Get list of key of both source data and target data
    source_key = get_key_list(source_data)
    target_key = get_key_list(target_data)

    # Find different key for add_new_key operation
    diff_key = sorted(list(set(target_key).difference(set(source_key))))
    # Remove different key in case of new dictionary
    index = 0
    while index < len(diff_key)-1:
        if diff_key[index] in diff_key[index+1]:
            diff_key.pop(index+1)
            index -= 1
        index += 1
    
    # Find same key existed in both data for modified_value_operation
    intersect_key = sorted(list(set(source_key).intersection(set(target_key))))

    # Union all keys in order to sort the operation
    checked_key = sorted(list(set(diff_key).union(set(intersect_key))))
    
    # Start log operation
    for key in checked_key:
        # Define operation
        operation = "add_new_key" if key in diff_key else "modify_value"

        # Find parent key list
        separated_key = key.split(".")
        parent_key = separated_key[0:-1]

        # Find after key strin
        after_key = get_after_key(target_key, key)

        # Find target_value and source_value with excluding dictionary value in case of modified
        if key in intersect_key:
            target_value = get_values(target_data, key, exclude_dict_value=True)
            source_value = get_values(source_data, key, exclude_dict_value=True)
        
        # Get normally value if it's add_new_key ops and get different value (target value) if it's modified ops
        value = get_values(target_data, key) if key in diff_key else get_different_value(source_value, target_value)
        
        # Append the dictionary of log
        output.append(log(operation, value, parent_key, after_key)) if value is not None else output
    return output

# Extract data source

In [20]:
source_data = extract_json_data(path, source_name)
target_data = extract_json_data(path, target_name)

# Process the operation

In [21]:
all_operation = []
all_operation += get_operation(source_data, target_data)

# Print the result

In [22]:
all_operation

[{'operation': 'add_new_key',
  'value': {'date': '2024-08-10'},
  'parent_key': ['event'],
  'after_key': 'name'},
 {'operation': 'modify_value',
  'value': {'country': 'Japan'},
  'parent_key': ['event', 'location'],
  'after_key': 'city'},
 {'operation': 'add_new_key',
  'value': {'venue': {'name': 'Tokyo Dome', 'capacity': 55000}},
  'parent_key': ['event', 'location'],
  'after_key': 'country'},
 {'operation': 'add_new_key',
  'value': {'artist': 'Alt-J'},
  'parent_key': ['event', 'performances', 'supporting', 'second'],
  'after_key': None}]